In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

from video      import read_qcif, save_qcif, height, width
from tools      import benchmark
from Difference_image import Difference_image_compute, Difference_image_reverse
from LZW        import LZW_encode, LZW_decode
from Huffman    import Huffman_encode, Huffman_decode
from DCT        import DCT_compute, DCT_inverse


# Format for qcif images
width = 176
height = 144
frame_size = width*height + 2*width//2*height//2

videos_names = []
for fname in os.listdir("./videos"):
    if fname.endswith(".qcif"):
        videos_names.append("./videos/"+fname)
print(videos_names)

['./videos/coastguard.qcif', './videos/hall.qcif', './videos/news.qcif', './videos/akiyo.qcif', './videos/carphone.qcif']


In [4]:
for fname in videos_names[0:1]:
    frames = read_qcif(fname)
    nb_frames = len(frames)
    
    diff = Difference_image_compute(frames)
    decompressed_data = Difference_image_reverse(diff)
    print()
    print("#--------------------------------------#")
    print("Diff on "+fname)
    print("MSE: ", np.mean(abs(frames-decompressed_data)))
    print("Original size (MiB): ", frames.size/1024**2)
    print()
    print("#--------------------------------------#")
    print("Diff+LZW on "+fname)
    bitstream = LZW_encode(diff.flatten())
    decompressed_diff = LZW_decode(bitstream)
    decompressed_data = Difference_image_reverse(np.array(decompressed_diff,dtype=float).reshape(nb_frames,-1))
    benchmark(frames,bitstream)
    print()
    print("#--------------------------------------#")
    print("Diff+Huffman on "+fname)
    bitstream = Huffman_encode(diff.flatten())
    decompressed_diff = Huffman_decode(bitstream)
    decompressed_data = Difference_image_reverse(np.array(decompressed_diff,dtype=float).reshape(nb_frames,-1))
    benchmark(frames,bitstream)
    print("\n\n")


#--------------------------------------#
Diff on ./videos/coastguard.qcif
MSE:  0.0
Original size (MiB):  10.87646484375

#--------------------------------------#
Diff+LZW on ./videos/coastguard.qcif


Decoding: 100%|██████████| 2472692/2472692 [00:12<00:00, 204663.47it/s]


B/symb:  4.769857603815937
Original size (MiB):  10.87646484375
Compressed size (%):  59.62322004769921

#--------------------------------------#
Diff+Huffman on ./videos/coastguard.qcif


Decoding: 100%|██████████| 51512688/51512688 [00:46<00:00, 1116043.67it/s]


B/symb:  4.517092627665544
Original size (MiB):  10.87646484375
Compressed size (%):  56.463657845819306





In [3]:
for fname in videos_names[0:1]:
    frames = read_qcif(fname)
    nb_frames = len(frames)
    
    dct = DCT_compute(frames,Q='opti')
    decompressed_data = DCT_inverse(dct)
    print()
    print("#--------------------------------------#")
    print("DCT+quantization+runlength on "+fname)
    print("MSE: ", np.mean(abs(frames-decompressed_data)))
    print("Original size (MiB): ", frames.size/1024**2)
    print("Compressed size (%): ", dct.size/frames.size*100)
    print()
    print("#--------------------------------------#")
    print("DCT+quantization+runlength+Huffman on "+fname)
    bitstream = LZW_encode(np.array(dct.flatten()))
    decompressed_dct  = LZW_decode(bitstream)
    decompressed_data = DCT_inverse(decompressed_dct).reshape(nb_frames,-1)
    benchmark(frames,bitstream)
    print()
    print("#--------------------------------------#")
    print("DCT+quantization+runlength+Huffman on "+fname)
    bitstream = Huffman_encode(np.array(dct.flatten()))
    decompressed_dct = Huffman_decode(bitstream)
    decompressed_data = DCT_inverse(decompressed_dct).reshape(nb_frames,-1)
    benchmark(frames,bitstream)
    print("\n\n")

DCT: 100%|██████████| 300/300 [02:13<00:00,  2.25it/s]


DCT: optimal Q as been set to  22.698961937716263 % of the psychovisual matrices values. 
[array([30])]


Building the dictionary:   0%|          | 0/7196334 [00:00<?, ?it/s]


#--------------------------------------#
DCT+quantization+runlength on ./videos/coastguard.qcif
MSE:  1.909783687570146
Original size (MiB):  10.87646484375
Compressed size (%):  63.099177539281705

#--------------------------------------#
DCT+quantization+runlength+Huffman on ./videos/coastguard.qcif


100%|██████████| 7196335/7196335 [04:33<00:00, 26332.20it/s]


B/symb:  2.3256020272166107
Original size (MiB):  10.87646484375
Compressed size (%):  29.070025340207632

#--------------------------------------#
DCT+quantization+runlength+Huffman on ./videos/coastguard.qcif


100%|██████████| 7196335/7196335 [04:32<00:00, 26378.57it/s]


B/symb:  2.4480457351290683
Original size (MiB):  10.87646484375
Compressed size (%):  30.600571689113355





In [5]:
for fname in videos_names[0:1]:
    frames = read_qcif(fname)
    nb_frames = len(frames)
    
    diff = Difference_image_compute(frames)
    dct = DCT_compute(diff,Q='opti')
    decompressed_diff = DCT_inverse(dct)
    decompressed_data = Difference_image_reverse(decompressed_diff)
    print((diff-decompressed_diff).max())
    print(abs(diff-decompressed_diff).mean())
    print((diff-decompressed_diff).min())
    print()
    print("#--------------------------------------#")
    print("Diff+DCT+quantization+runlength on "+fname)
    print("MSE: ", np.mean(abs(frames-decompressed_data)))
    print("Original size (MiB): ", frames.size/1024**2)
    print("Compressed size (%): ", dct.size/frames.size*100)
    print()
    print("#--------------------------------------#")
    print("Diff+DCT+quantization+runlength+Huffman on "+fname)
    bitstream = LZW_encode(np.array(dct.flatten()))
    decompressed_dct  = LZW_decode(bitstream)
    decompressed_diff = DCT_inverse(decompressed_dct).reshape(nb_frames,-1)
    decompressed_data = Difference_image_reverse(decompressed_diff)
    benchmark(frames,bitstream)
    print()
    print("#--------------------------------------#")
    print("Diff+DCT+quantization+runlength+Huffman on "+fname)
    #bitstream = Huffman_encode(np.array(dct.flatten()))
    #decompressed_dct = Huffman_decode(bitstream)
    #decompressed_diff = DCT_inverse(decompressed_dct).reshape(nb_frames,-1)
    #decompressed_data = Difference_image_reverse(decompressed_diff)
    #benchmark(frames,bitstream)
    print("\n\n")

DCT: 100%|██████████| 300/300 [02:17<00:00,  2.18it/s]


DCT: optimal Q as been set to  146.159169550173 % of the psychovisual matrices values. 


DCT inverse: 100%|██████████| 1524633/1524633 [04:27<00:00, 5695.65it/s]


86
4.0892219065656565
-223

#--------------------------------------#
Diff+DCT+quantization+runlength on ./videos/coastguard.qcif


Building the dictionary:   4%|▎         | 56627/1524632 [00:00<00:02, 566223.29it/s]

MSE:  79.57942918771043
Original size (MiB):  10.87646484375
Compressed size (%):  13.368344907407407

#--------------------------------------#
Diff+DCT+quantization+runlength+Huffman on ./videos/coastguard.qcif


DCT inverse: 100%|██████████| 1524633/1524633 [04:26<00:00, 5722.73it/s]


B/symb:  0.31067462822671155
Original size (MiB):  10.87646484375
Compressed size (%):  3.883432852833894

#--------------------------------------#
Diff+DCT+quantization+runlength+Huffman on ./videos/coastguard.qcif





In [5]:
if False:
    image_id = 2 # Random image from the video
    sep1 = height*width
    sep2 = sep1+height//2*width//2


    original_img     = frames[image_id][:sep1].reshape(height,width)
    decompressed_img = decompressed_data[image_id,:sep1].reshape(height,width)

    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(original_img)
    plt.colorbar()
    plt.title("Original")
    plt.subplot(1,2,2)
    plt.imshow(decompressed_img)
    plt.colorbar()
    plt.title("Comp/decomp image")
    plt.show()

    original_img     = frames[image_id][sep1:sep2].reshape(height//2,width//2)
    decompressed_img = decompressed_data[image_id,sep1:sep2].reshape(height//2,width//2)

    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(original_img)
    plt.colorbar()
    plt.title("Original")
    plt.subplot(1,2,2)
    plt.imshow(decompressed_img)
    plt.colorbar()
    plt.title("Comp/decomp image")
    plt.show()

    original_img     = frames[image_id][sep2:].reshape(height//2,width//2)
    decompressed_img = decompressed_data[image_id,sep2:].reshape(height//2,width//2)

    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(original_img)
    plt.colorbar()
    plt.title("Original")
    plt.subplot(1,2,2)
    plt.imshow(decompressed_img)
    plt.colorbar()
    plt.title("Comp/decomp image")
    plt.show()

In [7]:
save_qcif("test_res.qcif",decompressed_data)

0 255
